Importing dependencies

In [59]:
import pandas as pd
import datetime
import numpy as np

Importing Data

In [60]:
pd.set_option("display.precision", 2)
pd.options.display.float_format = '{:20,.2f}'.format
raw_data = pd.read_csv(r'BC_LCG_2017_CN.csv')
additional_info_1 = pd.read_excel(r'BC_LCG_2017_CN.xlsb', sheet_name='IA1', index_col=0)
additional_info_2 = pd.read_excel(r'BC_LCG_2017_CN.xlsb', sheet_name='IA2',index_col=0)

In [61]:
#checking data
cleaned = raw_data
cleaned.dropna(how='all', axis=0, inplace=True)
cleaned.head()

,Fecha,Número de Vendedor,Unnamed: 2,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo,Unnamed: 12,Unnamed: 13
0,1/01/2015,7.00,NaN,NaN,66.00,Real,21.00,NaN,328.00,TRATAMIENTO DE CONCRETO,"31,740.12","22,046.54",NaN,NaN
1,1/01/2015,5.00,NaN,NaN,45.00,Real,21.00,NaN,370.00,PINTURAS DECORATIVAS,"2,377.30","1,761.35",NaN,NaN
2,1/01/2015,8.00,NaN,NaN,75.00,Real,NaN,Pinturas,371.00,BARNICES ARQUITECTÓNICOS,"20,422.48","13,189.05",NaN,NaN
3,1/01/2015,6.00,NaN,NaN,57.00,Real,21.00,NaN,372.00,PINTURA AUTOMOTRIZ,719.80,668.40,NaN,NaN
4,1/01/2015,7.00,NaN,NaN,62.00,Real,21.00,NaN,376.00,TINTES PARA MADERA,"15,589.78","10,835.03",NaN,NaN


In [62]:
#creating dict for additional info
add_inf_dict1 = {}
for a,b in additional_info_1.iterrows():
    add_inf_dict1[b.name] = b.Departamento

add_inf_dict1_inverse = {v: k for k, v in add_inf_dict1.items()}
print(add_inf_dict1)
print(add_inf_dict1_inverse)

add_inf_dict2 = {}
for a,b in additional_info_2.iterrows():
    add_inf_dict2[b.name] = b.Nombre + ' ' + b.Apellido

{15: 'Alfombras', 806: 'Artículos Impulso', 7: 'Artículos Navideños', 25: 'Automotriz', 8: 'Baños', 11: 'Bebes', 2: 'Blancos', 5: 'Cocina', 42: 'Cortinaje', 2250: 'Decoracion', 22: 'Eléctricos', 3: 'Electrodomésticos', 58: 'Electronica', 26: 'Ferretería', 2251: 'Guatemala nuestra', 23: 'Herramientas', 27: 'Jardinería', 1: 'Juguetes', 10: 'Lámparas', 12: 'Librería', 30: 'Limpieza', 2248: 'Mascotas', 50: 'Materiales de Construcción', 4: 'Mesa', 59: 'Muebles de Exterior', 2249: 'Muebles de Interior', 807: 'Niños', 9: 'Organización', 28: 'Out Door Living', 21: 'Pinturas', 24: 'Plomería', 2247: 'Seguridad'}
{'Alfombras': 15, 'Artículos Impulso': 806, 'Artículos Navideños': 7, 'Automotriz': 25, 'Baños': 8, 'Bebes': 11, 'Blancos': 2, 'Cocina': 5, 'Cortinaje': 42, 'Decoracion': 2250, 'Eléctricos': 22, 'Electrodomésticos': 3, 'Electronica': 58, 'Ferretería': 26, 'Guatemala nuestra': 2251, 'Herramientas': 23, 'Jardinería': 27, 'Juguetes': 1, 'Lámparas': 10, 'Librería': 12, 'Limpieza': 30, 'Masco

Adding missing data and droppig empty rows/columns

In [63]:
#seller names
seller_names = []
for code in cleaned['Número de Vendedor']:
    seller_names.append(add_inf_dict2[code])
cleaned['Nom_Completo_Vendedor'] = seller_names
cleaned.dropna(how='all', axis=1, inplace=True)
cleaned = cleaned.where(pd.notnull(cleaned), '0')
cleaned['Departamento - Clave'] = cleaned['Departamento - Clave'].astype('int')
cleaned.head()

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo
0,1/01/2015,7,Carlos Vela,66,Real,21,0,328,TRATAMIENTO DE CONCRETO,"31,740.12","22,046.54"
1,1/01/2015,5,Gonzalo Aguilar,45,Real,21,0,370,PINTURAS DECORATIVAS,"2,377.30","1,761.35"
2,1/01/2015,8,Ernesto Pineda,75,Real,0,Pinturas,371,BARNICES ARQUITECTÓNICOS,"20,422.48","13,189.05"
3,1/01/2015,6,Paulina Casanova,57,Real,21,0,372,PINTURA AUTOMOTRIZ,719.80,668.40
4,1/01/2015,7,Carlos Vela,62,Real,21,0,376,TINTES PARA MADERA,"15,589.78","10,835.03"


In [64]:
#departmento
dept_codes = list(cleaned['Departamento - Clave'])
dept_names = list(cleaned['Departamento'])
dept_names_new = []
#print('Len codes: ',len(dept_codes))
#print('Len names: ',len(dept_names))

for i, row in enumerate(dept_codes):
    if row == 0:
        try: 
            dept_codes[i] = add_inf_dict1_inverse[dept_names[i]]
        except:
            print('Exception at Code index: ', i, row)
            dept_codes[i] = 'Inf. faltante'

for row in dept_codes:
    if row == 'Inf. faltante':
        dept_names_new.append('Inf. faltante')
    else:
        dept_names_new.append(add_inf_dict1[row])

#print('Len codes: ',len(dept_codes))
#print('Len names: ',len(dept_names_new))

cleaned['Departamento - Clave'] = dept_codes
cleaned['Departamento'] = dept_names_new
cleaned['Ventas_USD'] = cleaned['Ventas Netas (Q)']/7.5
cleaned['Costo_USD'] = cleaned['Costo']/7.5
cleaned['Ganancias_USD'] = cleaned['Ventas_USD'] - cleaned['Costo_USD']
cleaned.head()

,Fecha,Número de Vendedor,Nom_Completo_Vendedor,Número de cliente,Tipo,Departamento - Clave,Departamento,Familia - Clave,Familia,Ventas Netas (Q),Costo,Ventas_USD,Costo_USD,Ganancias_USD
0,1/01/2015,7,Carlos Vela,66,Real,21,Pinturas,328,TRATAMIENTO DE CONCRETO,"31,740.12","22,046.54","4,232.02","2,939.54","1,292.48"
1,1/01/2015,5,Gonzalo Aguilar,45,Real,21,Pinturas,370,PINTURAS DECORATIVAS,"2,377.30","1,761.35",316.97,234.85,82.13
2,1/01/2015,8,Ernesto Pineda,75,Real,21,Pinturas,371,BARNICES ARQUITECTÓNICOS,"20,422.48","13,189.05","2,723.00","1,758.54",964.46
3,1/01/2015,6,Paulina Casanova,57,Real,21,Pinturas,372,PINTURA AUTOMOTRIZ,719.80,668.40,95.97,89.12,6.85
4,1/01/2015,7,Carlos Vela,62,Real,21,Pinturas,376,TINTES PARA MADERA,"15,589.78","10,835.03","2,078.64","1,444.67",633.97


PARETO ABC 

DEPARTAMENTOS

In [65]:
#Departamentos
#1.1 Crear Dataset de Departamentos
df_departamentos = cleaned.groupby('Departamento').agg(
    clientes = ('Número de cliente', 'nunique'),
    pedidos = ('Familia', 'nunique'),
    ingresos = ('Ventas_USD', 'sum'),
    ganancias = ('Ganancias_USD', 'sum'),
).reset_index()

df_departamentos.head()

,Departamento,clientes,pedidos,ingresos,ganancias
0,Alfombras,40,3,"3,382,987.11","1,729,639.42"
1,Artículos Impulso,41,4,"4,201,449.06","1,635,760.79"
2,Artículos Navideños,59,23,"7,772,727.44","3,997,732.89"
3,Automotriz,80,11,"10,472,857.66","3,994,262.15"
4,Baños,53,6,"12,903,730.01","6,218,372.94"


In [66]:
#1.2 Segmentación ABC
def abc(df, metric_column, abc_class_name='class'):
    """Asigna una clase ABC y ranking dentro de la clase de acuerdo con la metrica de contribución 
        al porcentaje cumulativo 
    
    Args:
        df: Pandas dataframe containing data. 
        metric_column (string): Name of column containing metric to calculate. 
        abc_class_name (string, optional): Name to assign to class column. 
    
    Return:
        Pandas dataframe containing original data, plus the metric class and rank. 
    """
    
    def _abc_segment(percentage):
        """Asigna una clase ABC de acuerdo con la metrica de contribución al porcentaje cumulativo 
        Args:
            percentage (float): Cumulative percentage of ranked metric.
        Returns:
            segments: Pandas DataFrame
        """

        if 0 < percentage <= 80:
            return 'A'
        elif 80 < percentage <= 90:
            return 'B'
        else:
            return 'C' 
    data = df.sort_values(by=metric_column, ascending=False)
    data[metric_column+'_sum'] = data[metric_column].sum()
    data[metric_column+'_cumsum'] = data[metric_column].cumsum()
    data[metric_column+'_running_pc'] = (data[metric_column+'_cumsum'] / data[metric_column+'_sum']) * 100
    data[abc_class_name] = data[metric_column+'_running_pc'].apply(_abc_segment)
    data[abc_class_name+'_rank'] = data[metric_column+'_running_pc'].rank().astype(int)
    data.drop([metric_column+'_sum', metric_column+'_cumsum', metric_column+'_running_pc'], axis=1, inplace=True)
    return data

df_departamento_segmentos = abc(df_departamentos, 'ingresos', 'pareto_abc')
df_departamento_segmentos.head()

,Departamento,clientes,pedidos,ingresos,ganancias,pareto_abc,pareto_abc_rank
17,Juguetes,80,28,"20,274,844.59","8,080,787.56",A,1
12,Eléctricos,83,15,"13,653,840.73","6,060,140.03",A,2
7,Cocina,84,18,"13,540,891.31","6,268,931.74",A,3
4,Baños,53,6,"12,903,730.01","6,218,372.94",A,4
10,Electrodomésticos,70,16,"12,539,779.33","4,766,885.59",A,5


In [72]:
#1.3 Analysis ABC para examinar los segmentos
df_resumen_abc_departamentos = df_departamento_segmentos.groupby('pareto_abc').agg(
    departamentos = ('Departamento', 'nunique'),
    pedidos = ('pedidos', 'sum'),
    ingresos = ('ingresos', 'sum'),
    ganancias = ('ganancias', 'sum'),
).reset_index()

df_resumen_abc_departamentos.head()


,pareto_abc,departamentos,pedidos,ingresos,ganancias
0,A,18,262,"184,530,954.91","77,521,031.81"
1,B,5,89,"26,639,366.45","11,659,704.24"
2,C,9,50,"23,582,028.68","10,722,938.41"


CLIENTES

In [73]:
#Departamentos
#2.1 Crear Dataset de Clientes
df_clientes = cleaned.groupby('Número de cliente').agg(
    cliente = ('Número de cliente', 'nunique'),
    pedidos = ('Familia', 'nunique'),
    ingresos = ('Ventas_USD', 'sum'),
    ganancias = ('Ganancias_USD', 'sum'),
).reset_index()

df_clientes.head()

,Número de cliente,cliente,pedidos,ingresos,ganancias
0,1,1,62,"2,726,283.87","1,151,872.49"
1,2,1,54,"1,953,236.54","778,842.68"
2,3,1,56,"1,468,666.56","601,176.17"
3,4,1,50,"1,321,047.86","563,615.86"
4,5,1,52,"1,467,000.73","615,561.38"


In [74]:
#2.2 Segmentación ABC
df_cliente_segmentos = abc(df_clientes, 'ingresos', 'pareto_abc')
df_cliente_segmentos.head()

,Número de cliente,cliente,pedidos,ingresos,ganancias,pareto_abc,pareto_abc_rank
46,47,1,283,"20,530,094.33","8,314,814.64",A,1
13,14,1,273,"19,963,858.34","8,940,207.06",A,2
58,59,1,296,"19,198,607.44","8,321,570.14",A,3
67,68,1,292,"18,611,336.47","8,172,306.51",A,4
73,74,1,284,"16,949,299.52","7,147,048.04",A,5


In [76]:
#2.3 Analysis ABC para examinar los segmentos
df_resumen_abc_clientes = df_cliente_segmentos.groupby('pareto_abc').agg(
    clientes = ('cliente', 'nunique'),
    pedidos = ('pedidos', 'sum'),
    ingresos = ('ingresos', 'sum'),
    ganancias = ('ganancias', 'sum'),
).reset_index()

df_resumen_abc_departamentos.head()

,pareto_abc,departamentos,pedidos,ingresos,ganancias
0,A,18,262,"184,530,954.91","77,521,031.81"
1,B,5,89,"26,639,366.45","11,659,704.24"
2,C,9,50,"23,582,028.68","10,722,938.41"


VENDEDORES

In [83]:
#3.1 Tabla de resumen de ventas por vendedor
df_vendedores = cleaned.groupby('Nom_Completo_Vendedor').agg(
    cliente = ('Número de cliente', 'nunique'),
    pedidos = ('Familia', 'nunique'),
    ingresos = ('Ventas_USD', 'sum'),
    ganancias = ('Ganancias_USD', 'sum'),
).reset_index()
df_vendedores = df_vendedores.sort_values('ingresos', ascending=False)
df_vendedores['Porcentajes_ventas'] = df_vendedores['ingresos']/df_vendedores['ingresos'].sum()
df_vendedores['Porcentajes_clientes'] = df_vendedores['cliente']/df_vendedores['cliente'].sum()
df_vendedores['Porcentajes_pedidos'] = df_vendedores['pedidos']/df_vendedores['pedidos'].sum()

df_vendedores



,Nom_Completo_Vendedor,cliente,pedidos,ingresos,ganancias,Porcentajes_ventas,Porcentajes_clientes,Porcentajes_pedidos
3,Juan Garza,87,366,"126,053,855.59","53,692,083.69",0.54,0.55,0.18
1,Ernesto Pineda,16,262,"27,277,862.38","11,569,402.83",0.12,0.10,0.13
2,Gonzalo Aguilar,9,251,"15,621,483.76","6,721,964.72",0.07,0.06,0.13
7,Sofia Gonzalez,10,257,"15,238,778.79","6,472,645.19",0.06,0.06,0.13
0,Carlos Vela,9,220,"14,234,057.78","6,133,075.64",0.06,0.06,0.11
5,Paulina Casanova,9,215,"14,146,193.68","5,992,204.63",0.06,0.06,0.11
6,Pedro Arroyo,10,259,"13,928,986.07","6,053,357.49",0.06,0.06,0.13
4,Michelle Perez,9,177,"8,251,131.99","3,268,940.26",0.04,0.06,0.09


Analisis de Tendencia de Ventas